# Analyze social media posts

Setting up sap version of LangChain (sap-llm-commons). Instructions are adapted from https://github.tools.sap/AILearning/genAIDay. To follow these instructions, VPM c|onnection is required. **Need to ask what is at the moment the reccomendation for non-sap employees**

In [ ]:
#!pip install "sap-llm-commons===0.1.0" --extra-index-url https://int.repositories.cloud.sap/artifactory/api/pypi/proxy-deploy-releases-hyperspace-pypi/simple --trusted-host int.repositories.cloud.sap
#!pip install langchain==0.0.281 openai==0.28.1 panel tiktoken pypdf chromadb pycryptodome lark wikipedia DateTime jupyter_bokeh langchain docarray django==2.2 django-cors-headers==2.5.3 django-storages-redux==1.3.3 selenium==4.8.2 panel
#!pip install ai-api-client-sdk
#!pip install ai-core-sdk

In [6]:
pip show sap_llm_commons

Name: sap-llm-commons
Version: 0.1.0
Summary: 
Home-page: 
Author: Deep Learning Center of Excellence
Author-email: xxx@global.corp.sap
License: 
Location: /Users/I550349/Library/CloudStorage/OneDrive-SAPSE/Documents/workspace/py_envs/hana_vector_env/lib/python3.9/site-packages
Requires: 
Required-by: 
Note: you may need to restart the kernel to use updated packages.


After downloading llm-commons you need to create a folder named .aicore_llm in your home directory and inside you need to create a config.json file like this:

```
{
    "AICORE_LLM_AUTH_URL": " ... ",
    "AICORE_LLM_CLIENT_ID": " ... ",
    "AICORE_LLM_CLIENT_SECRET": " ... ",
    "AICORE_LLM_API_BASE": " ... ",
    "AICORE_LLM_RESOURCE_GROUP": " ... "
}
```
You can create automatically the file by downloading the access key of your AI Core instance and typing in the command line: 
```
aicore-llm configure -k your-aicore-access-key.txt
```

### Let's test if we can use llm_commons

In [1]:
from llm_commons.proxy.base import set_proxy_version
set_proxy_version('aicore') # for an AI Core proxy
from llm_commons.langchain.proxy import ChatOpenAI
llm = ChatOpenAI(  proxy_model_name='gpt-4' ,\
                    temperature=0.0,\
                    verbose = True)
llm.predict('What is your name?')

"I am an AI language model developed by OpenAI, and I don't have a personal name. You can simply refer to me as OpenAI Assistant. How can I assist you today?"

In [4]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

info_dict={ "category": '''Identify if the social media reports a situation related to one of the following categories: 
                             1. PUBLIC CLEANLINESS: Dirty public areas, overflowing dustbins and littering. Bulky waste in common areas.  
                             2. ROADS & FOOTPATHS: Including covered linkways, signboards & streetlights. E.g. Pot holes, huge cracks, etc.
                             3. FACILITY & PARK MAINTENANCE: Fallen trees, overgrown grass, and maintenance of park lighting and facilities.
                             4. PESTS: Sighting of bees and hornets, potential mosquito breeding sites, and more.
                             5. DRAINS & SEWERS: Choked, overflowing, or damaged drains, bad sewage smells, flooding.   
                             Output the category name. If none of the categories fits, or in doubt, return OTHER - PLEASE CHECK.  
                             ''',                 
            "priority": '''Identify the priority to be given to the reported issues: 
                                      
                             1. LOW: the issue does not pose any problem with public safety and does not necessarily need to be handled urgently. \
                             2. MEDIUM: the issue does not cause any immediate danger, but it has significant and negative impact on the daily life of people in the neighborhood.\
                             3. HIGH: the issue needs to be resolved quickly because it can potentially cause dangerous situations or \
                                 disruptions. 
                             4. VERY HIGH: the issue needs to be handled as soon as possible, as it is a matter of public safety. \
                             
                             Return the priority level. If in doubt, return MEDIUM ''',          
            "title": "Summarize the issue that is being reported in 40 characters and a neutral tone.",
            "description": "Summarize the issue that is being reported in not more that 300 characters and a neutral tone.",

            "location":"Extract the address where the issue has been noticed. Return the street only and omit the town or country",
            "coordinates": "Extract the coordinates where the issue has been notices. The format should be: (float, float)",
       
            "sentiment" : '''Extract the sentiment of the post: 
                            1. NEUTRAL: if the issue is reported politely
                            2. NEGATIVE: if the post shows irritation, impatience, annoyance
                            3. VERY NEGATIVE: the post expresses rage, hatred
                            '''
 }

category_schema = ResponseSchema(name="category",
                             description=info_dict['category'])
priority_schema = ResponseSchema(name="priority",
                                      description=info_dict['priority'])
title_schema = ResponseSchema(name="title",
                                    description=info_dict['title'])
description_schema = ResponseSchema(name="description",
                                    description=info_dict['description'])
location_schema= ResponseSchema(name="location",
                                    description=info_dict['location'])
coordinates_schema= ResponseSchema(name="coordinates",
                                    description=info_dict['coordinates'], type='tuple')
sentiment_schema= ResponseSchema(name="sentiment",
                                    description=info_dict['sentiment'])

response_schemas = [category_schema, 
                    priority_schema,
                    title_schema,
                    description_schema,
                    location_schema,
                    coordinates_schema,
                    sentiment_schema,                   
                    ]

output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()

In [21]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import SimpleJsonOutputParser

info_dict={ "category": '''Identify if the social media reports a situation related to one of the following categories: 
                             1. PUBLIC CLEANLINESS: Dirty public areas, overflowing dustbins and littering. Bulky waste in common areas.  
                             2. ROADS & FOOTPATHS: Including covered linkways, signboards & streetlights. E.g. Pot holes, huge cracks, etc.
                             3. FACILITY & PARK MAINTENANCE: Fallen trees, overgrown grass, and maintenance of park lighting and facilities.
                             4. PESTS: Sighting of bees and hornets, potential mosquito breeding sites, and more.
                             5. DRAINS & SEWERS: Choked, overflowing, or damaged drains, bad sewage smells, flooding.   
                             Output the category name. If none of the categories fits, or in doubt, return OTHER - PLEASE CHECK.  
                             ''',                 
            "priority": '''Identify the priority to be given to the reported issues: 
                                      
                             1. LOW: the issue does not pose any problem with public safety and does not necessarily need to be handled urgently. \
                             2. MEDIUM: the issue does not cause any immediate danger, but it has significant and negative impact on the daily life of people in the neighborhood.\
                             3. HIGH: the issue needs to be resolved quickly because it can potentially cause dangerous situations or \
                                 disruptions. 
                             4. VERY HIGH: the issue needs to be handled as soon as possible, as it is a matter of public safety. \
                             
                             Return the priority level. If in doubt, return MEDIUM ''',          
            "title": "Summarize the issue that is being reported in 40 characters and a neutral tone.",
            "description": "Summarize the issue that is being reported in not more that 300 characters and a neutral tone.",

            "location":"Extract the address where the issue has been noticed. Return the street only and omit the town or country",
            "coordinates": "Extract the coordinates where the issue has been notices. The format should be: (float, float)",
       
            "sentiment" : '''Extract the sentiment of the post: 
                            1. NEUTRAL: if the issue is reported politely
                            2. NEGATIVE: if the post shows irritation, impatience, annoyance
                            3. VERY NEGATIVE: the post expresses rage, hatred
                            '''
 }

category_schema = ResponseSchema(name="category",
                             description=info_dict['category'])
priority_schema = ResponseSchema(name="priority",
                                      description=info_dict['priority'])
title_schema = ResponseSchema(name="title",
                                    description=info_dict['title'])
description_schema = ResponseSchema(name="description",
                                    description=info_dict['description'])
location_schema= ResponseSchema(name="location",
                                    description=info_dict['location'])
coordinates_schema= ResponseSchema(name="coordinates",
                                    description=info_dict['coordinates'])
sentiment_schema= ResponseSchema(name="sentiment",
                                    description=info_dict['sentiment'])

response_schemas = [category_schema, 
                    priority_schema,
                    title_schema,
                    description_schema,
                    location_schema,
                    coordinates_schema,
                    sentiment_schema,                   
                    ]

output_parser = SimpleJsonOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()

ImportError: cannot import name 'SimpleJsonOutputParser' from 'langchain.output_parsers' (/Users/I550349/Library/CloudStorage/OneDrive-SAPSE/Documents/workspace/py_envs/hana_vector_env/lib/python3.9/site-packages/langchain/output_parsers/__init__.py)

In [5]:
format_instructions

'The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n```json\n{\n\t"category": string  // Identify if the social media reports a situation related to one of the following categories: \n                             1. PUBLIC CLEANLINESS: Dirty public areas, overflowing dustbins and littering. Bulky waste in common areas.  \n                             2. ROADS & FOOTPATHS: Including covered linkways, signboards & streetlights. E.g. Pot holes, huge cracks, etc.\n                             3. FACILITY & PARK MAINTENANCE: Fallen trees, overgrown grass, and maintenance of park lighting and facilities.\n                             4. PESTS: Sighting of bees and hornets, potential mosquito breeding sites, and more.\n                             5. DRAINS & SEWERS: Choked, overflowing, or damaged drains, bad sewage smells, flooding.   \n                             Output the category name. If none of th

In [13]:
my_post='''📢 Attention to all Sagenai residents! 🏡🌳

I hope this message finds you well. 😊 However, I have a matter of concern that I would like to share with you all. 🙏🏼 Recently, I noticed a distressing situation at the public area near High Road, Sagenai. 🚮🤢 The area is in dire need of cleaning and maintenance, as it has become terribly dirty and unkempt. This issue may not pose immediate danger, but it significantly impacts the quality of life for all of us in the neighborhood. 🙅🏻‍♀️🏘️

I kindly request the attention of our local authorities to address this problem promptly. 📞🏢 Let's work together to restore the cleanliness and beauty of our beloved neighborhood. 🌟🌷 Your timely action will be greatly appreciated! 🙌🏼💚

#Sagenai #CleanlinessMatters #CommunityUnity" 
 

        Coordinates:(51.5728533044047,-0.03229501754130372) '''

In [14]:
from langchain.prompts import ChatPromptTemplate

template = """\
For the following social media post, extract the following information:

{category}

{priority}

{title}

{description}

{location}

{coordinates}

{sentiment}

social media post: {text}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template=template)

messages = prompt.format_messages(text=my_post,
                                category=info_dict['category'],
                                priority=info_dict['priority'],
                                title=info_dict['title'],
                                description=info_dict['description'],
                                location=info_dict['location'],
                                coordinates=info_dict['coordinates'],
                                sentiment=info_dict['sentiment'],
                                format_instructions=format_instructions)

In [15]:
response = llm(messages)

In [16]:
output_dict = output_parser.parse(response.content)

In [18]:
type(output_dict['coordinates'])

str